<a href="https://colab.research.google.com/github/dantae74/web-crawling/blob/main/7_%EC%9E%90%EB%8F%99%ED%99%94_%EB%B0%8F_%EB%8B%A4%EC%9A%B4%EB%A1%9C%EB%93%9C_%EC%97%85%EB%A1%9C%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 자동화 및 다운로드/업로드

---

In [ ]:
# Install selenium
!pip install selenium

In [ ]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import requests
import os
import shutil
import datetime
import pandas as pd

## os 모듈 연습하기

In [ ]:
# 현재 작업 디렉토리 가져오기
cwd = os.getcwd()
print(f'현재 작업 디렉토리: {cwd}')


In [ ]:
# 현재 디렉토리에 있는 모든 파일과 디렉토리를 나열하기
print('현재 디렉토리의 파일 및 디렉토리:')
print(os.listdir(cwd))


In [ ]:

# 새 디렉토리 생성하기
os.mkdir('new_directory')
print('새 디렉토리 생성 후:')
print(os.listdir(cwd))


In [ ]:

# 디렉토리 이름 변경하기
os.rename('new_directory', 'renamed_directory')
print('디렉토리 이름 변경 후:')
print(os.listdir(cwd))


In [ ]:

# 디렉토리 제거하기
os.rmdir('renamed_directory')
print('디렉토리 제거 후:')
print(os.listdir(cwd))


## shutil 모듈 연습하기

In [ ]:

# 현재 작업 디렉토리를 확인하고, 그곳에 새로운 디렉토리와 파일을 생성합니다.
cwd = os.getcwd()
os.mkdir(os.path.join(cwd, 'dir1'))
with open(os.path.join(cwd, 'dir1', 'file1.txt'), 'w') as file:
    file.write('Hello, World!')


In [ ]:

# 'dir1' 디렉토리를 'dir2'로 복사합니다.
shutil.copytree(os.path.join(cwd, 'dir1'), os.path.join(cwd, 'dir2'))
print('디렉토리 복사 후 파일 목록:')
print(os.listdir(cwd))


In [ ]:

# 'dir2' 내의 'file1.txt'를 'file2.txt'로 복사합니다.
shutil.copy(os.path.join(cwd, 'dir2', 'file1.txt'), os.path.join(cwd, 'dir2', 'file2.txt'))
print('파일 복사 후 파일 목록:')
print(os.listdir(os.path.join(cwd, 'dir2')))


In [ ]:

# 'dir1' 디렉토리를 'dir3'로 이동(이름 변경)합니다.
shutil.move(os.path.join(cwd, 'dir1'), os.path.join(cwd, 'dir3'))
print('디렉토리 이동 후 파일 목록:')
print(os.listdir(cwd))


In [ ]:

# 'dir2'와 'dir3' 디렉토리를 제거합니다.
# 이 때, 디렉토리 내부에 파일이 있는 경우에도 제거가 가능합니다.
shutil.rmtree(os.path.join(cwd, 'dir2'))
shutil.rmtree(os.path.join(cwd, 'dir3'))
print('디렉토리 제거 후 파일 목록:')
print(os.listdir(cwd))

## 다운로드 자동화

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

In [ ]:
# -*- coding: UTF-8 -*-
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

- 네이버 뉴스 기사 크롤링에서 이미지 다운로드 경로 크롤링 추가

In [ ]:
def get_news_info(driver):
    result = []
    # 뉴스 기사들을 모아 놓은 요소가 로딩될때 까지 대기
    articles_box = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#main_pack > section > div > div.group_news > ul')))
    # 뉴스 기사 리스트 찾기
    article_list = articles_box.find_elements(By.CSS_SELECTOR,'li.bx')
    for article in article_list:
        news_info = {}
        # 뉴스 제목 가져오기
        title = article.find_element(By.CSS_SELECTOR,'a.news_tit').text
        news_info['제목'] = title
        # 뉴스 내용 가져오기
        dsc = article.find_element(By.CSS_SELECTOR, "div.dsc_wrap > a").text
        news_info['내용'] = dsc
        # 신문사 가져오기
        press = article.find_element(By.CSS_SELECTOR, "a.press").text
        news_info['언론사'] = press
        # 뉴스 이미지 경로 가져오기
        try:
            img = article.find_element(By.CSS_SELECTOR,'a.dsc_thumb > img')
            # href 속성 가져오기
            img_src = img.get_attribute('src')
        except:
            img_src = ""
        news_info['이미지 경로'] = img_src

        result.append(news_info)
    return result

def get_page_info(driver):
    result = {}
    # 페이지 이동마다 사용하기 때문에 Wait 사용
    page_wrap = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR , 'div.sc_page')))
    # 현재 페이지 찾기
    now_page = page_wrap.find_element(By.CSS_SELECTOR, 'a[aria-pressed="true"]').text
    result['현재 페이지'] = now_page
    # 다음 페이지 이동 불가능 여부 가져오기
    next_page = page_wrap.find_element(By.CSS_SELECTOR, 'a.btn_next')
    result['다음 페이지'] = next_page
    aria_disabled = next_page.get_attribute('aria-disabled')
    result['다음 페이지 이동 불가'] = aria_disabled


    return result

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

In [ ]:
# 드라이버 로드
# driver = webdriver.Chrome()

# 페이지 접근
driver.get('https://search.naver.com/search.naver?where=news&sm=tab_jum&query=KT WIZ') # 크롤링 예제 사이트

df = []

# 최대 10페이지까지 크롤링
for i in range(0,10):
    news_info = get_news_info(driver)
    page_info = get_page_info(driver)
    print(page_info['현재 페이지'])
    for news in news_info:
        df.append(news)
    # 다음 페이지로 이동
    if page_info['다음 페이지 이동 불가'] == 'true':
        break
    else:
        page_info['다음 페이지'].click()

driver.quit()

- 데이터 프레임으로 만들기

In [ ]:
df = pd.DataFrame(df)
df

- 오늘 날짜로 폴더 만들기

In [ ]:
# 오늘 날짜가 없다면 폴더 생성
# if not exists same name folder, create folder
date= datetime.datetime.now().strftime('%Y%m%d%H')
if not os.path.exists(date):
    os.mkdir(date)

In [ ]:
for idx, src in zip(df.index,df['이미지 경로']):
    if src != "" and src.startswith('http'):
        response = requests.get(src)
        with open(f"{date}/{idx}.png", "wb") as file:
            file.write(response.content)


In [ ]:
cwd = os.getcwd()
print(f'현재 작업 디렉토리: {cwd}')
print('현재 디렉토리의 파일 및 디렉토리:')
print(os.listdir(cwd))
print(os.listdir(os.path.join(cwd, '2023070307')))


## 업로드 자동화

- 다운로드/업로드 연습 사이트에서 업로드 실습 진행

In [ ]:
from selenium import webdriver

# driver = webdriver.Chrome()

options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)


driver.get("https://demoqa.com/upload-download")

In [ ]:
upload = driver.find_element(By.CSS_SELECTOR, "#uploadFile")
file_path = os.getcwd() + f"/{date}/0.png"

# send_keys() 메소드를 이용해 파일 업로드
upload.send_keys(file_path)

# 업로드 버튼 클릭
